In [4]:
import geoai

## Set Raster and Vector URLS 

In [5]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train.tif"
)
vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train_buildings.geojson"

## Download Raster and Vector Files

In [ ]:
raster_path = geoai.download_file(raster_url)


File already exists: naip_train.tif


In [7]:
vector_path = geoai.download_file(vector_url)

File already exists: naip_train_buildings.geojson


## Download Building Footprint Model (ESRI)

In [8]:
extractor = geoai.BuildingFootprintExtractor()

Model path not specified, downloading from Hugging Face...
Model downloaded to: /Users/aslanvaroqua/.cache/huggingface/hub/models--giswqs--geoai/snapshots/089548329c81f128fa12576663e7abdedb5cfa0e/building_footprints_usa.pth
Model loaded successfully


## Save Masks

In [9]:
mask_path = extractor.save_masks_as_geotiff(
    raster_path=raster_path,
    output_path="building_masks.tif",
    confidence_threshold=0.5,
    mask_threshold=0.5,
)

Processing masks with parameters:
- Confidence threshold: 0.5
- Chip size: (512, 512)
- Mask threshold: 0.5
Dataset initialized with 3 rows and 7 columns of chips
Image dimensions: 2503 x 1126 pixels
Chip size: 512 x 512 pixels
Overlap: 25.0% (stride_x=384, stride_y=384)
CRS: EPSG:26911
Processing raster with 6 batches


100%|██████████| 6/6 [00:16<00:00,  2.68s/it]

Object masks saved to building_masks.tif


## Convert Masks to Vector

In [10]:
gdf = extractor.masks_to_vector(
    mask_path=mask_path,
    output_path="building_masks.geojson",
    simplify_tolerance=1.0,
)

Converting mask to GeoJSON with parameters:
- Mask threshold: 0.5
- Min object area: 100
- Max object area: None
- Simplify tolerance: 1.0
- NMS IoU threshold: 0.5
- Regularize objects: True
- Angle threshold: 15° from 90°
- Rectangularity threshold: 70.0%
Mask dimensions: (1126, 2503)
Mask value range: 0 to 255
Found 696 potential objects


100%|██████████| 696/696 [00:00<00:00, 1090.97it/s]


Created 664 valid polygons
Object count after NMS filtering: 664
Regularizing 664 objects...
- Angle threshold: 15° from 90°
- Min orthogonality: 30.0% of angles
- Min rectangularity: 70.0% of bounding box area


100%|██████████| 664/664 [00:00<00:00, 3522.04it/s]

Regularization completed:
- Total objects: 664
- Rectangular objects: 660 (99.4%)
- Other regularized objects: 0 (0.0%)
- Unmodified objects: 4 (0.6%)


Saved 664 objects to building_masks.geojson


## Process Raster in One Step

In [11]:
gdf = extractor.process_raster(
    raster_path,
    output_path="buildings.geojson",
    batch_size=4,
    confidence_threshold=0.5,
    overlap=0.25,
    nms_iou_threshold=0.5,
    min_object_area=100,
    max_object_area=None,
    mask_threshold=0.5,
    simplify_tolerance=1.0,
)

Processing with parameters:
- Confidence threshold: 0.5
- Tile overlap: 0.25
- Chip size: (512, 512)
- NMS IoU threshold: 0.5
- Mask threshold: 0.5
- Min object area: 100
- Max object area: None
- Simplify tolerance: 1.0
- Filter edge objects: True
- Edge buffer size: 20 pixels
Dataset initialized with 3 rows and 7 columns of chips
Image dimensions: 2503 x 1126 pixels
Chip size: 512 x 512 pixels
Overlap: 25.0% (stride_x=384, stride_y=384)
CRS: EPSG:26911
Processing raster with 6 batches


100%|██████████| 6/6 [00:17<00:00,  2.98s/it]


Objects before filtering: 749
Objects after filtering: 749
Saved 749 objects to buildings.geojson


## Regularize the polygons 

In [12]:
gdf_regularized = extractor.regularize_buildings(
    gdf=gdf,
    min_area=100,
    angle_threshold=15,
    orthogonality_threshold=0.3,
    rectangularity_threshold=0.7,
)

Regularizing 749 objects...
- Angle threshold: 15° from 90°
- Min orthogonality: 30.0% of angles
- Min rectangularity: 70.0% of bounding box area


100%|██████████| 749/749 [00:00<00:00, 4047.84it/s]

Regularization completed:
- Total objects: 749
- Rectangular objects: 696 (92.9%)
- Other regularized objects: 0 (0.0%)
- Unmodified objects: 53 (7.1%)


## Visualize

In [13]:



gdf.head()


,geometry,confidence,class
548,"POLYGON ((455272.2 5277763.2, 455271 5277762, ...",0.999945,1
1234,"POLYGON ((455984.4 5277643.8, 455983.8 5277643...",0.999941,1
888,"POLYGON ((456074.4 5277719.4, 456073.2 5277718...",0.999941,1
549,"POLYGON ((455243.4 5277855, 455242.8 5277854.4...",0.999938,1
438,"POLYGON ((454989 5277799.2, 454988.4 5277798.6...",0.999931,1


In [14]:

geoai.view_vector_interactive(    gdf, column="confidence", layer_name="Building", tiles="Satellite")


In [15]:

geoai.view_vector_interactive(    gdf, column="confidence", layer_name="Building", tiles=raster_url)


In [16]:

geoai.view_vector_interactive(    gdf_regularized, column="confidence", layer_name="Building", tiles=raster_url)

 